In [81]:
##############################
## CREATE COUNTABLE BINARY ARRAY with field view !!!!!!
#############################

import numpy as np
import scipy.misc as smp
import random



arr_size_half = 20
max_N = 1
view_size_half = 1

def create_inp_outp_array():
    max_objects = 3
    n_ones = random.randint(1, max_objects)
    one_positions = random.sample(range(view_size_half, view_size_half + max_objects), n_ones)
    binary_array = np.zeros(max_objects+2*view_size_half)
    binary_array[one_positions] = 1
    binary_array[0] = 5
    binary_array[-1] = 5

    return binary_array, n_ones
    

 

def make_data_set(n_samples):
  mult_inp, mult_out = create_inp_outp_array()
  for i in range(n_samples - 1):
      single_inp, single_out = create_inp_outp_array()
      mult_inp = np.vstack([mult_inp, single_inp])
      mult_out = np.vstack((mult_out, single_out))
  return mult_inp, mult_out



train_input, train_output = make_data_set(1000)
max_N=20
test_input, test_output = make_data_set(1000)


#Print single one
inp_ex, out_ex = make_data_set(10)
print("inp_ex", inp_ex)
print("out_ex", out_ex)


#Print set data
print("===========================")
print("train_input: number of examples: ", train_input.shape)
print("train_input: number of examples: ", train_input[:,0].size)
print("train_input: single example size: ", train_input[0].size)
print("train_input: dim of elements in example: ", train_input[0,0].size)




('inp_ex', array([[5., 1., 0., 0., 5.],
       [5., 1., 1., 1., 5.],
       [5., 0., 1., 0., 5.],
       [5., 0., 1., 1., 5.],
       [5., 1., 0., 1., 5.],
       [5., 0., 1., 0., 5.],
       [5., 1., 0., 1., 5.],
       [5., 1., 1., 1., 5.],
       [5., 1., 1., 0., 5.],
       [5., 1., 1., 1., 5.]]))
('out_ex', array([[1],
       [3],
       [1],
       [2],
       [2],
       [1],
       [2],
       [3],
       [2],
       [3]]))
('train_input: number of examples: ', (1000, 5))
('train_input: number of examples: ', 1000)
('train_input: single example size: ', 5)
('train_input: dim of elements in example: ', 1)


In [0]:
########################################
## UPDATE ENVIRONMENT WHEN MOVING, PICKING WITH LIMITED FIELD VIEW
##################################################################


def update_env(state, pos, action, reward):
  
  push_finish_button = False
  #reward=0.0
  if(action==0):        #[N+2] ----> pick the one at the current position
    if(state[pos]==1):
      state[pos]= 0
      reward += point_reward
  elif(action==1):        #[N+3] ----> stop the current counting process
    push_finish_button = True
  elif(action==2):          #[N] ----> go to the left
    if(pos>view_size_half):
      pos-=1
  elif(action==3):        #[N+1] ----> go to the right
     if(pos<state.size - view_size_half-1):
      pos+=1
  else:
    ("How could your action go beyond N+3?")
    
  return state, pos, push_finish_button, reward 
    
    
def get_field_view(state, pos, view_size_half):
  fiew_field =  copy.copy(state[pos-view_size_half:pos+view_size_half+1])
  return fiew_field

def action_int_to_letter(action):
  
  if(action==0):          #[N] ----> go to the left
     act_str = "P"
  elif(action==1):        #[N+1] ----> go to the Right
     act_str = "STOP"
  elif(action==2):        #[N+2] ----> pick the one at the current position
    act_str = "L"
  elif(action==3):        #[N+3] ----> stop the current counting process
    act_str = "R"
  else:
    ("How could your action go beyond N+3?")
    
  return act_str
    

def get_animation_pic(state,pos,view_size_half,action):
  s = " "
  for i in range(pos-view_size_half):
    s+="   "
  s += "|"
  for i in range(view_size_half):
    s+="__"
  s += "I"
  for i in range(view_size_half):
    s+="__"
  s += "|"
  for i in range(state.size - (pos-view_size_half+3) ):
    s+="   "
  s+= "  "
  act_str = action_int_to_letter(action)
  s+= act_str
  print(state)
  print(s)
  

In [83]:
#!/usr/bin/env python

#import gym
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
#from gym.envs.classic_control.cartpole import CartPoleEnv
#from cartpole_utils import plot_results,print_results
import tensorflow as tf
import copy
import matplotlib
import matplotlib.pyplot as plt


def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01)/ np.sqrt(dim_state), name=name)


def Q_network(state_holder, w_h, w_h2, w_o, b):
    h = tf.nn.relu(tf.matmul(state_holder, w_h))
    #h2 = tf.nn.relu(tf.matmul(h, w_h2))
    Q = tf.matmul(h, w_o, name='output_activation') + b
    #Q = tf.nn.sigmoid(a_z, name='Q_model')
    return Q


########################
## SET PARAMETERS
########################
# Algorithm parameters

point_reward = +10.0
time_penalty_reward = -0.2
final_reward = +0.0
stop_reward = +0.0



learning_rate = 3e-3
gamma = 1.0
epsilon = 1.
epsi_decay = .995
lr_decay = .995
epsilon_min = 0.01
learning_min = 0.00001


# Load Input=Start-of-a-game &&  Classes=Check-for-reward
trX = train_input
trY = train_output
teX = test_input
teY = test_output

print(trX.shape)
print(trY.shape)
print(teX.shape)
print(teY.shape)


## SET ENVIRONMENT + #ofActions #ofStates
dim_state = trX[0,:].size
n_action = 4     #[Pick, Stop, Left, Right]
dim_obs = 1+2*view_size_half
N_print_every = 100









##############################
## BUILD TENSORFLOW
#############################


# PLACEHOLDER
action_holder = tf.placeholder(dtype=tf.int32, name="Y")  # +1 because that is the node responsible to say if it is done ore not
state_holder = tf.placeholder(dtype=tf.float32, shape=(1, dim_obs), name='symbolic_state')
next_state_holder = tf.placeholder(dtype=tf.float32, shape=(1, dim_obs), name='symbolic_state')

# Initialize the parameters of the Q model
n_h_neurons = 20
w_h = init_weights([dim_obs, n_h_neurons], "w_h")
w_h2 = init_weights([n_h_neurons, n_h_neurons], "w_h2")
w_o = init_weights([n_h_neurons, n_action], "w_o")
b0 = np.zeros(n_action, dtype=np.float32)
b = tf.Variable(initial_value=b0, trainable=True, name='bias')

Q = Q_network(state_holder, w_h, w_h2, w_o,b)
next_Q = Q_network(next_state_holder, w_h, w_h2, w_o,b)


### COMPARE REWARDS ####
#PLACEHOLDER
r_holder = tf.placeholder(dtype=tf.float32, name='symbolic_value_estimation')
is_done_holder = tf.placeholder(dtype=tf.float32, name='is_done')

next_action_Q_max = tf.to_int32(tf.argmax(next_Q, dimension=1, name='next_step_maxQ_action'))           # [ 0.1   0.4   0.8  0.8] -->  3, 4
R = Q[0, action_holder]
next_R = r_holder + gamma * next_Q[0, next_action_Q_max[0]] * (1 - is_done_holder)
error = (R - next_R)**2

# Define the operation that performs the optimization
learning_rate_holder = tf.placeholder(dtype=tf.float32, name='symbolic_state')
training_step = tf.train.GradientDescentOptimizer(learning_rate_holder).minimize(error)

sess = tf.Session()  # FOR NOW everything is symbolic, this object has to be called to compute each value of Q
sess.run(tf.initialize_all_variables())

################################
################################





################################
## PI(S)
## First check if its random time THEN
        # - calculate State--Network--> Q
        # - take maximum of Q
        # - just remember and return index of maximum    ---> returns ACTION that leads to max_Q-value (not value itself)
################################

def policy(state):
    if rd.rand() < epsilon:
        return rd.randint(0, n_action)
    reshaped_state = state.reshape(1, dim_state)
    Q_values = sess.run(Q, feed_dict={state_holder: state.reshape(1, dim_state)})
    Q_values_reduced = Q_values[0, :]
    val = np.max(Q_values[0, :])                                                                                      #Change in dimension
    max_indices = np.where(Q_values[0, :] == val)[0]
    push_finish_button = False

    return rd.choice(max_indices), push_finish_button




########################
## Get maximum Q_VALUE (not action)
#######################
def Q_max(state):
    Q_values = sess.run(Q, feed_dict={state_holder: state.reshape(1, dim_state)})
    return Q_values.max()






##############################
## ACTUAL RUN
###########################

time_list = []
reward_list = []
err_list = []
val_list = []
mean_reward = 0
print_every_n = 50
trial_duration = 15

mean_reward_list = []
n_episodes = trX[:, 0].size
for k in range(n_episodes):

    acc_reward = 0  # Init the accumulated reward
    push_finish_button = False
    
    k_n = rd.randint(0, trX[:,0].size)                                            ######   !!!!!!!!!!!!!!!!!

    if(epsilon>epsilon_min):
        epsilon *= epsi_decay
    if (learning_rate > learning_min):
        learning_rate *= lr_decay

    state = copy.copy(trX[k_n, :])
    pos = view_size_half
    observation = get_field_view(state, pos, view_size_half)

    trial_err_list = []
    internal_counter_memory = 0
    
    
    for t in range(trial_duration):  # The number of time steps in this game is maximum 200
        

        old_observation = copy.copy(observation)
        old_internal_counter_memory = copy.copy(internal_counter_memory)
        state_copy = copy.copy(state)

        #####################
        ## Chose action by Random or Maximum Q-value
        ################################
        if rd.rand() < epsilon:
          action = rd.randint(0, n_action)
        else:
          Q_values = sess.run(Q, feed_dict={state_holder: observation.reshape(1, dim_obs)})
          Q_values_reduced = Q_values[0, :]
          val = np.max(Q_values_reduced)  # Change in dimension
          max_indices = np.where(Q_values_reduced == val)[0]
          action = rd.choice(max_indices)


        ######################
        ## Update Environment, get reward according to chosen action
        #################################################
        reward = 0     
        new_state, pos, push_finish_button, reward = update_env(state_copy, pos, action,reward)
        new_observation = get_field_view(new_state, pos, view_size_half)

        if(push_finish_button==False):
          reward += time_penalty_reward
        
        #print("Check train_step vars")
        #print("observation", observation)
        #print("new_observation", new_observation)
        #print("action", action_int_to_letter(action))
        #print("Reward", reward)
        
        ############################
        ## TRAINING STEP, ERROR CALCULATION (RUN SESSION)
        ############################
        err = sess.run(error, feed_dict={
            state_holder: observation.reshape(1, dim_obs),
            next_state_holder: new_observation.reshape(1, dim_obs),
            action_holder: action,
            is_done_holder: np.float32(push_finish_button),
            r_holder: reward})

        sess.run(training_step, feed_dict={
            state_holder: observation.reshape(1, dim_obs),
            next_state_holder: new_observation.reshape(1, dim_obs),
            action_holder: action,
            is_done_holder: np.float32(push_finish_button),
            r_holder: reward,
            learning_rate_holder: learning_rate})

        observation = new_observation  # Pass the new state to the next step
        old_state = copy.copy(state)
        state = new_state
        acc_reward += reward  # Accumulate the reward
        trial_err_list.append(err)

        
        ########################
        ## PRINT IN SINGLE TIME STEPS
        #########################
        

             
        if(k>=100 and k<105):
          if(t==0):
            print("\n ================")
            print("K= ", k)
          print("t: ", t)
          print("Action", action)
          get_animation_pic(state,pos,view_size_half,action)
          #print("observation: ", observation)
          #print("new_observation: ", new_observation)
          print("Reward: ", reward)
          print("--------------")
 
        
        if (push_finish_button==True or t==trial_duration-1):
          mean_reward += acc_reward
          break
        

    ########################
    ## PRINT MEAN AFTER EVERY N TIME STEPS
    ###################################
    if (k % print_every_n == 0):
        mean_reward = float(mean_reward) / print_every_n
        print("K", k)
        print(k, "th mean_reward=", mean_reward)
        print("Epsilon: ", epsilon)
        mean_reward_list.append(mean_reward)
        mean_reward = 0.0
         

    err_list.append(np.mean(trial_err_list))
    time_list.append(t + 1)
    reward_list.append(acc_reward)  # Store the result


print("mean_reward_list", mean_reward_list)
reward_file = 'Mean_rewards/'  + str(n_h_neurons) + '_neurons_' + str(n_action) + '_max_objects'
np.save(reward_file, mean_reward_list)


matplotlib.get_backend()
legend_string = str(n_h_neurons) + " neurons/hidden layer"
plt.plot(np.arange(0,len(mean_reward_list)), mean_reward_list, label = legend_string)
plt.legend()
print("legend_string= ", legend_string)
plt.ylabel('Success rate')
plt.xlabel('Episodes')
plt.ylim(0,1)
title_string = 'Success rate for agent when counting binaries between 0 and ' + str(n_action) + ' binaries'
plt.title(title_string)
plt.savefig('Q_learning_binaries.png')
plt.show()


(1000, 5)
(1000, 1)
(1000, 5)
(1000, 1)
('K', 0)
(0, 'th mean_reward=', 0.16000000000000006)
('Epsilon: ', 0.995)
('K', 50)
(50, 'th mean_reward=', 4.024000000000001)
('Epsilon: ', 0.7744209942832988)

('K= ', 100)
('t: ', 0)
('Action', 1)
[5. 0. 0. 1. 5.]
 |__I__|        STOP
('Reward: ', 0)
--------------
('K', 100)
(100, 'th mean_reward=', 6.016000000000002)
('Epsilon: ', 0.6027415843082742)

('K= ', 101)
('t: ', 0)
('Action', 2)
[5. 0. 1. 1. 5.]
 |__I__|        L
('Reward: ', -0.2)
--------------
('t: ', 1)
('Action', 0)
[5. 0. 1. 1. 5.]
 |__I__|        P
('Reward: ', -0.2)
--------------
('t: ', 2)
('Action', 2)
[5. 0. 1. 1. 5.]
 |__I__|        L
('Reward: ', -0.2)
--------------
('t: ', 3)
('Action', 0)
[5. 0. 1. 1. 5.]
 |__I__|        P
('Reward: ', -0.2)
--------------
('t: ', 4)
('Action', 2)
[5. 0. 1. 1. 5.]
 |__I__|        L
('Reward: ', -0.2)
--------------
('t: ', 5)
('Action', 3)
[5. 0. 1. 1. 5.]
    |__I__|     R
('Reward: ', -0.2)
--------------
('t: ', 6)
('Action', 0)

IOError: ignored

In [68]:
a=5
b=a+2
print("b",b)
a=7
print("b",b)

('b', 7)
('b', 7)
